In [8]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
import numpy as np
import math
import pandas as pd
import matplotlib
%matplotlib inline
import sys
import os
import glob
import re

# janelas = [2,3,4,5]
janelas = [3]

for k in janelas:

    abordagemAA = '2 - AA[dg-arj]' #'2 - AA[pet-chr]' ou '2 - AA[dg-arj]'
    dirOrig = "../csv/conjuntosDS/treinoTeste-k"+str(k)+"/"
    # dirOrig = "c:/Users/Diego/Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
    # dirOrig = "../arqResult/csv/conjuntosDS/conjuntosDiverg/treinoTeste/"
    # estat = "../csv/estatisticaInicialDS2.csv"
    estat = "../csv/estatisticaInicialDS2-DgArj-k"+str(k)+".csv"
    # estat = "c:/Users/Diego/Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/estatisticaInicialDS2.csv"
    # estat = "../arqResult/csv/estatisticaInicialDS2.csv"

    print(dirOrig)
    
    etapa = '3 - clasf'

    # estatisticas = pd.read_csv(estat, index_col=['etapa','permutacao'], sep=';')
    estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados','permutacao','etapa'], sep=';')
#     estatisticas.tail()

    arquivos = os.listdir(dirOrig)
    # print(arquivos)
    #print(len(arquivos))

    train = []
    test = []

    for arq in arquivos:
        if arq.startswith("train"):
            train.append(arq)
        elif arq.startswith("test"):
            test.append(arq)

    # print('tamanho de train: %d' %(len(train)))        

    # print('tamanho de test: %d' %(len(test)))

    # print (train)    
    # print (test)    



    #lista = ['10', '15', '20', '25'] #Assumindo que serão esses os conjuntos de algoritmos
    lista = ['10', '15', '20'] #Assumindo que serão esses os conjuntos de algoritmos
#     lista = ['10'] #Assumindo que serão esses os conjuntos de algoritmos
    # lista = ['3'] #Assumindo que serão esses os conjuntos de algoritmos

    for i in lista: 

        cont = 0

        trainAtual = []
        testAtual = []

        for arq in train:
            if '('+i+')' in arq:
#             if 'train(10)22' in arq:    
                trainAtual.append(arq)
        for arq in test:
            if '('+i+')' in arq:
#             if 'test(10)22' in arq:
                testAtual.append(arq)

    #     print('lista desordenada')
    #     print(trainAtual)

    #     print('tamanho antes: %d' %(len(trainAtual)))

        #Ordenando a lista
        trainAtual = sorted(trainAtual)
        testAtual = sorted(testAtual)

    #     for x in trainAtual:
    #         print(x)



    # #     print('lista ordenada')
    # #     print(trainAtual)

    #     print('tamanho depois: %d' %(len(trainAtual)))

    #     print(type(trainAtual))


        print (i)

    #     print(trainAtual)
    #     print(testAtual)

        tam = len(trainAtual) #Mesma coisa para testAtual

        for pos in range(tam):
    #         print(pos)
            print('')
            print('##########################')
            print('Arquivos atuais: %s e %s' %(trainAtual[pos], testAtual[pos]))
            print('##########################')

            treino = dirOrig+trainAtual[pos]
            teste = dirOrig+testAtual[pos]

    #         print('treino: %s' %(treino))

    #         tp, fp, tn, fn = 0

            num = trainAtual[pos].replace('train('+i+')','')
            num = num.replace('.csv','')

            cont += 1

            print('Iteração: %d' %(cont))

    #         print('num: %s' %(num))

            algUtl = re.sub('train.*\(', r'', trainAtual[pos]) #Alterar para fazer a substituição de tudo em uma linha só
            algUtl = re.sub('\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
            algUtl = int(algUtl)


            print('algUtl: %d' %(algUtl))

            permutacao = int(num)
    #         linhaAtual = estatisticas.loc[('2 - AA[pet-chr]', permutacao), : ] #Armazena a linha correspondente à permutação
    #         linhaAtual = estatisticas.loc[('1 - acm diverg', permutacao), : ] #Armazena a linha correspondente à permutação
            linhaAtual = estatisticas.loc[algUtl, permutacao, abordagemAA, : ] #Armazena a linha correspondente à permutação
#             print (linhaAtual)
#             print(linhaAtual.columns)

        #'''
    #         all = pd.read_csv("train.csv", index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'duplicata'), header = None)
    #         toClass = pd.read_csv("test.csv", index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'duplicata'), header = None)

    #         all = pd.read_csv(treino, index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'duplicata'), header = None)
    #         toClass = pd.read_csv(teste, index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'duplicata'), header = None)

            all = pd.read_csv(treino, index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'track10', 'track11', 'duplicata'), header = 0) #Era header = None
            toClass = pd.read_csv(teste, index_col=False, sep=';', names=('title', 'artist', 'track01', 'track02', 'track03', 'track10', 'track11', 'duplicata'), header = 0) #Era header = None


            # all['artist-title'] = all['artist'] * all['title']
            # cols = list(all.columns.values)
            # cols.pop(cols.index('duplicata'))
            # all = all[cols+['duplicata']]

            #Pesos baseados no Alg17

            all['artist-title'] = all['artist'] * all['title']
            all['soma-pesos'] = (all['artist']*1 + all['title']*2 + all['track01']*0.8 + all['track02']*0.8)/4.6 + (all['track03'] + all['track10'] + all['track11'])/3
#             all['tracks'] = ((all['track01'] * all['track02'] * all['track03'] * all['track10'] * all['track11'])/(all['artist'] + all['title']))
            cols = list(all.columns.values)
            cols.pop(cols.index('duplicata'))
            all = all[cols+['duplicata']]

            # toClass['artist-title'] = toClass['artist'] * toClass['title']
            # cols = list(toClass.columns.values)
            # cols.pop(cols.index('duplicata'))
            # toClass = toClass[cols+['duplicata']]

            #Pesos baseados no Alg17

            toClass['artist-title'] = toClass['artist'] * toClass['title']
            toClass['soma-pesos'] = (toClass['artist']*1 + toClass['title']*2 + toClass['track01']*0.8 + toClass['track02']*0.8)/4.6 + (toClass['track03'] + toClass['track10'] + toClass['track11'])/3
#             toClass['tracks'] = ((toClass['track01'] * toClass['track02'] * toClass['track03'] * toClass['track10'] * toClass['track11'])/(toClass['artist'] + toClass['title']))
            cols = list(toClass.columns.values)
            cols.pop(cols.index('duplicata'))
            toClass = toClass[cols+['duplicata']]

            #Separação do conjunto X do conjunto y
            # X = all.loc[:,'title':'artist-title']
            X = all.loc[:,'title':'soma-pesos']
            y = all.duplicata

            # XtoClass = toClass.loc[:,'title':'artist-title']
            XtoClass = toClass.loc[:,'title':'soma-pesos']
            ytoClass = toClass.duplicata

            from sklearn import model_selection

            X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state=2)

            from sklearn.ensemble import RandomForestClassifier
            from sklearn.model_selection import StratifiedKFold
            from sklearn.model_selection import KFold
            from sklearn import model_selection

            seed = 500

            '''

            from sklearn.linear_model import LogisticRegression
            from sklearn.tree import DecisionTreeClassifier
            from sklearn.neighbors import KNeighborsClassifier
            from sklearn.svm import SVC

            modelos = []
            modelos.append(('LR', LogisticRegression()))
            modelos.append(('KNN', KNeighborsClassifier()))
            modelos.append(('RFC', DecisionTreeClassifier()))
            modelos.append(('SVM', SVC()))
            modelos.append(('RF', RandomForestClassifier()))



            # Avaliação de cada modelo por vez
            resultados = []
            nomes = []
            for nome, modelo in modelos:
                kfold = StratifiedKFold(n_splits=10, random_state=seed) #Mudar para n-fold
                cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
                msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std()) #Iprimir em um arquivo
                print(msg)
            '''

            rfc = RandomForestClassifier(random_state=12)

            from sklearn.model_selection import GridSearchCV

            param_grid = [{"criterion": ['gini', 'entropy'],"n_estimators": [10,25,30,35,40,45,50,100],
                           #"max_depth": [2,3,4,5,6,7],
                          "class_weight": ["balanced",None],
                          "min_samples_split": [2, 5, 10]}]

            score = 'f1'

            print("# Tunando hiperparâmetros para %s" % score)
            print()

            grid = GridSearchCV(rfc, param_grid, cv=5, scoring='%s_macro' % score)
            grid.fit(X_train, y_train)

            print("Melhor conjunto de hiperparâmetros encontrado:")
            print()
            print(grid.best_params_)
            # print()
            # print("Grade de valores encontrados:")
            # print()
            medias = grid.cv_results_['mean_test_score']
            desvios = grid.cv_results_['std_test_score']
            # for media, desvio, params in zip(medias, desvios, grid.cv_results_['params']):
            #     print("%0.3f (+/-%0.03f) for %r" % (media, desvio * 2, params))
            #     print()

            rfc = RandomForestClassifier(**grid.best_params_, random_state=500)

    #         rfc = RandomForestClassifier(parameters = grid.best_params_, random_state=12)

#             kfold = KFold(n_splits=2, random_state=seed)
    
#             try:
    
#                 kfold = StratifiedKFold(n_splits=10, random_state=seed) #Mudar para n-fold #Adicionado depois!
        
#             except ValueError:
                
# #                 kfold = StratifiedKFold(n_splits=5, random_state=seed) #Tirei pra ver se melhora
            
# #             except ValueError:
                
#                 kfold = KFold(n_splits=10, random_state=seed)
            
# #             except ValueError:
                
# #                 kfold = KFold(n_splits=5, random_state=seed)
                
# #             except ValueError:
                
# #                 kfold = KFold(random_state=seed) #Default n_splits = 3
            
# #             except ValueError:
                
# #                 kfold = KFold(n_splits=2, random_state=seed)
            
#             except:
#                 print('Erro com a geração dos folds!')
#                 print('Erro com o arquivo: ' + treino)
#                 exc_type, exc_obj, exc_tb = sys.exc_info()
#                 fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#                 print(exc_type, fname, exc_tb.tb_lineno)
#                 print(sys.exc_info())
                
            #Isso só serve para validar o uso do conjunto de treino 
            #Não influencia no resultado final
#             try:
#                 cv_results2 = model_selection.cross_val_score(rfc, X_train, y_train, cv=kfold, scoring='f1')
#                 msg = "%s com hiperparâmetros tunados: F1 = %f com desvio padrão = %f" % ('RFC', cv_results2.mean(), cv_results2.std())
#                 print(msg)
            
#             except:
#                 print("ERRO NA VALIDAÇÃO CRUZADA!")
#                 exc_type, exc_obj, exc_tb = sys.exc_info()
#                 fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#                 print(exc_type, fname, exc_tb.tb_lineno)
#                 print(sys.exc_info())

                
            rfc.fit(X_train, y_train) #Treinando o modelo
            predicoes = rfc.predict(X_test) #Realizando a predição

    #         from sklearn.metrics import confusion_matrix
    #         matriz = confusion_matrix(y_test, predicoes)

    #         print(matriz)

            from sklearn.metrics import classification_report
            from sklearn.metrics import precision_recall_fscore_support

            # print(classification_report(y_test, predicoes))
    #         prec, rec, fbeta, supp = precision_recall_fscore_support(y_test, predicoes, average=None)
    #         print ('precision: %.5f' %(prec[1]))
    #         print ('recall: %.5f' %(rec[1]))
    #         print ('f1: %.5f' %(fbeta[1]))
    #         print ('non-matches: %d - matches: %d' %(supp[0], supp[1]))
    #         print('')

            # from sklearn.externals.six import StringIO  
            # from IPython.display import Image  
            # from sklearn.tree import export_graphviz
            # import pydotplus
            # dot_data = StringIO()
            # export_graphviz(rfc, out_file=dot_data,  
            #                 filled=True, rounded=True,
            #                 special_characters=True, feature_names=X.columns)
            # graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
            # Image(graph.create_png())

            predicoes = rfc.predict(XtoClass)

            from sklearn.metrics import confusion_matrix
            matriz = confusion_matrix(ytoClass, predicoes)
            print(matriz)

            this_tn, this_fp, this_fn, this_tp = 0, 0, 0, 0

            this_tn, this_fp, this_fn, this_tp = confusion_matrix(ytoClass, predicoes).ravel()
            print('tn, fp, fn, tp')
            print(this_tn, this_fp, this_fn, this_tp)



            from sklearn.metrics import classification_report
            from sklearn.metrics import precision_recall_fscore_support

            # print(classification_report(y_test, predicoes))
    #         prec, rec, fbeta, supp = precision_recall_fscore_support(ytoClass, predicoes, average=None)
    #         print ('precision: %.5f' %(prec[1]))
    #         print ('recall: %.5f' %(rec[1]))
    #         print ('f1: %.5f' %(fbeta[1]))
    #         print ('non-matches: %d - matches: %d' %(supp[0], supp[1]))
    #         print('')


            ########################

            #Atualizar as estatísticas
            abordagem = 'DS'
            #print 'abordagem é %s' %(abordagem)

            #algUtl = linhaAtual['algoritmosUtilizados'].item()
            iteracao = 2
            inspecoesManuais = linhaAtual['inspecoesManuais'].item()
            da = linhaAtual['da'].item() + (this_tp + this_fp) #Adiciona todos os casos positivos identificados
            dm = linhaAtual['dm'].item()             #na predição
            ndm = linhaAtual['ndm'].item()


            fn = float(linhaAtual['fn'].item() - this_tp ) #Não tenho certeza se é isso
            tp = this_tp + float(linhaAtual['tp'].item()) #+ dm) #Recuperar de toClass
            print('tp é igual a: %d' %(tp))

            fp = this_fp + float(linhaAtual['fp'].item())
            #tn = tn + float(linhaAtual['tn'].item() - tp) #+ ndm) #Recuperar de toClass
            tn = (9762 * 9763) / 2 -(tp+fp+fn)



            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            fmeasure = 2*((precision*recall)/(precision+recall))

            #Adicionando valor à última linha
    #         estatisticas.loc[(etapa, permutacao), ['abordagem', 'algoritmosUtilizados', 'iteracao', 'inspecoesManuais',
    #            'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
    #            'fp', 'tn', 'fn'] ] = ([abordagem, algUtl, iteracao, inspecoesManuais,
    #            precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])
            estatisticas.loc[(algUtl, permutacao, etapa), ['abordagem', 'iteracao', 'inspecoesManuais',
                'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais,
                precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])

            #'''

    # estatisticas = estatisticas.reset_index(level=['etapa', 'permutacao'])
    estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'permutacao', 'etapa'])
    # estatisticas.head()                   
    #                    

    estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

    estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
    estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

    dirEst = "../csv/"
    # dirEst = "c:/Users/Diego/Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/"
    # dirEst = "../arqResult/csv/"


    estatisticas.to_csv(dirEst+'estatisticaInicialDS3-k'+str(k)+'.csv', sep=';', index=False)        

../csv/conjuntosDS/treinoTeste-k3/
10

##########################
Arquivos atuais: train(10)10.csv e test(10)10.csv
##########################
Iteração: 1
algUtl: 10
# Tunando hiperparâmetros para f1

Melhor conjunto de hiperparâmetros encontrado:

{'class_weight': 'balanced', 'criterion': 'entropy', 'min_samples_split': 2, 'n_estimators': 35}
[[368   7]
 [ 13 145]]
tn, fp, fn, tp
368 7 13 145
tp é igual a: 242

##########################
Arquivos atuais: train(10)11.csv e test(10)11.csv
##########################
Iteração: 2
algUtl: 10
# Tunando hiperparâmetros para f1

Melhor conjunto de hiperparâmetros encontrado:

{'class_weight': 'balanced', 'criterion': 'gini', 'min_samples_split': 10, 'n_estimators': 10}
[[786   7]
 [ 20  94]]
tn, fp, fn, tp
786 7 20 94
tp é igual a: 244

##########################
Arquivos atuais: train(10)12.csv e test(10)12.csv
##########################
Iteração: 3
algUtl: 10
# Tunando hiperparâmetros para f1

Melhor conjunto de hiperparâmetros encontrado:



[[805  19]
 [ 12 165]]
tn, fp, fn, tp
805 19 12 165
tp é igual a: 253
20

##########################
Arquivos atuais: train(20)10.csv e test(20)10.csv
##########################
Iteração: 1
algUtl: 20
# Tunando hiperparâmetros para f1

Melhor conjunto de hiperparâmetros encontrado:

{'class_weight': 'balanced', 'criterion': 'gini', 'min_samples_split': 5, 'n_estimators': 45}
[[781  13]
 [  7 157]]
tn, fp, fn, tp
781 13 7 157
tp é igual a: 259

##########################
Arquivos atuais: train(20)11.csv e test(20)11.csv
##########################
Iteração: 2
algUtl: 20
# Tunando hiperparâmetros para f1

Melhor conjunto de hiperparâmetros encontrado:

{'class_weight': None, 'criterion': 'entropy', 'min_samples_split': 5, 'n_estimators': 25}
[[811  20]
 [ 11 174]]
tn, fp, fn, tp
811 20 11 174
tp é igual a: 254

##########################
Arquivos atuais: train(20)12.csv e test(20)12.csv
##########################
Iteração: 3
algUtl: 20
# Tunando hiperparâmetros para f1

Melhor conjunto de